## Install Everything

In [1]:
!pip install -q requests pandas sqlite-utils python-dotenv numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.3 MB/s eta 0:00:00


In [2]:
import os
import json
import requests
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timezone
from typing import Dict, List, Optional, Any, Union
from dotenv import load_dotenv
import time
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
def setup_environment():
    """Setup environment variables for Metaculus API"""

    # Create .env file content
    env_content = """# Metaculus Tracker Configuration
# Get your API token from: https://www.metaculus.com/settings/
METACULUS_API_TOKEN=9944b09199c62bcf9418ad846dd0e4bbdfc6ee4b

# API Endpoints
METACULUS_API_URL=https://www.metaculus.com/api
METACULUS_API_VERSION=v1

# Database Configuration
DB_PATH=/content/metaculus_tracker.db

# Tracking Configuration
REQUESTS_PER_HOUR=1000
MIN_FORECASTERS=10
FETCH_COMMENTS=true
FETCH_FORECASTS=true
"""

    # Write .env file
    env_path = Path("/content/.env")
    env_path.write_text(env_content)

    print(f"✅ Environment configuration created!")
    print(f"📁 Environment file created at: {env_path}")
    print("\n⚠️ IMPORTANT: Edit /content/.env and add your Metaculus API token")
    print("   Get your token from: https://www.metaculus.com/settings/")
    print("   Look for 'API Access' section")

    # Load environment variables
    load_dotenv(env_path)

    return {
        "api_url": os.getenv("METACULUS_API_URL", "https://www.metaculus.com/api"),
        "api_token": os.getenv("METACULUS_API_TOKEN"),
        "db_path": os.getenv("DB_PATH", "/content/metaculus_tracker.db")
    }

# Execute environment setup
env_config = setup_environment()


✅ Environment configuration created!
📁 Environment file created at: /content/.env

⚠️ IMPORTANT: Edit /content/.env and add your Metaculus API token
   Get your token from: https://www.metaculus.com/settings/
   Look for 'API Access' section


In [4]:
def setup_database():
    """Create SQLite database with all required tables"""

    db_path = os.getenv("DB_PATH", "/content/metaculus_tracker.db")
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create Posts table (equivalent to Events)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS posts (
        id INTEGER PRIMARY KEY,
        title TEXT,
        short_title TEXT,
        slug TEXT UNIQUE,
        author_id INTEGER,
        author_username TEXT,
        status TEXT,
        resolved BOOLEAN,
        published_at TEXT,
        actual_close_time TEXT,
        scheduled_close_time TEXT,
        scheduled_resolve_time TEXT,
        open_time TEXT,
        nr_forecasters INTEGER,
        comment_count INTEGER,
        vote_score INTEGER,
        forecasts_count INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)

    # Create Questions table (equivalent to Markets)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS questions (
        id INTEGER PRIMARY KEY,
        post_id INTEGER,
        title TEXT,
        type TEXT,  -- binary, numeric, multiple_choice, date
        status TEXT,
        resolution TEXT,
        resolution_criteria TEXT,
        fine_print TEXT,
        open_time TEXT,
        scheduled_close_time TEXT,
        actual_close_time TEXT,
        scheduled_resolve_time TEXT,
        actual_resolve_time TEXT,
        options TEXT,  -- JSON for multiple choice options
        scaling TEXT,  -- JSON for numeric scaling parameters
        aggregation_data TEXT,  -- JSON for latest aggregation
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (post_id) REFERENCES posts(id)
    )
    """)

    # Create Conditional Questions table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS conditional_questions (
        id INTEGER PRIMARY KEY,
        parent_post_id INTEGER,
        condition_id INTEGER,
        condition_child_id INTEGER,
        question_yes_id INTEGER,
        question_no_id INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (parent_post_id) REFERENCES posts(id),
        FOREIGN KEY (condition_id) REFERENCES questions(id),
        FOREIGN KEY (condition_child_id) REFERENCES questions(id)
    )
    """)

    # Create Group Questions table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS group_questions (
        id INTEGER PRIMARY KEY,
        parent_post_id INTEGER,
        group_variable TEXT,
        graph_type TEXT,
        description TEXT,
        resolution_criteria TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (parent_post_id) REFERENCES posts(id)
    )
    """)

    # Create Forecasts table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS forecasts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        question_id INTEGER,
        forecaster_id INTEGER,
        forecaster_username TEXT,
        start_time TEXT,
        end_time TEXT,
        forecaster_count INTEGER,
        probability_yes REAL,  -- For binary questions
        forecast_values TEXT,  -- JSON for continuous/multiple choice
        continuous_cdf TEXT,  -- JSON for continuous CDF
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (question_id) REFERENCES questions(id)
    )
    """)

    # Create Comments table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS comments (
        id INTEGER PRIMARY KEY,
        post_id INTEGER,
        author_id INTEGER,
        author_username TEXT,
        parent_id INTEGER,
        root_id INTEGER,
        text TEXT,
        vote_score INTEGER,
        included_forecast BOOLEAN,
        created_at TEXT,
        fetched_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (post_id) REFERENCES posts(id)
    )
    """)

    # Create Projects/Categories table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS projects (
        id INTEGER PRIMARY KEY,
        name TEXT,
        slug TEXT,
        type TEXT,
        description TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)

    # Create Post_Projects junction table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS post_projects (
        post_id INTEGER,
        project_id INTEGER,
        PRIMARY KEY (post_id, project_id),
        FOREIGN KEY (post_id) REFERENCES posts(id),
        FOREIGN KEY (project_id) REFERENCES projects(id)
    )
    """)

    # Create Aggregations table for tracking prediction aggregates
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS aggregations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        question_id INTEGER,
        aggregation_type TEXT,  -- metaculus_prediction, recency_weighted, unweighted
        start_time TEXT,
        end_time TEXT,
        forecaster_count INTEGER,
        centers TEXT,  -- JSON array
        interval_lower_bounds TEXT,  -- JSON array
        interval_upper_bounds TEXT,  -- JSON array
        forecast_values TEXT,  -- JSON for continuous
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (question_id) REFERENCES questions(id)
    )
    """)

    conn.commit()
    conn.close()

    print(f"✅ Database setup complete at: {db_path}")
    print("📊 Tables created:")
    print("   - posts (main questions/events)")
    print("   - questions (individual forecasting questions)")
    print("   - conditional_questions, group_questions")
    print("   - forecasts, comments, projects")
    print("   - aggregations (community predictions)")

# Execute database setup
setup_database()

✅ Database setup complete at: /content/metaculus_tracker.db
📊 Tables created:
   - posts (main questions/events)
   - questions (individual forecasting questions)
   - conditional_questions, group_questions
   - forecasts, comments, projects
   - aggregations (community predictions)


## Fetch and Store Active Questions/Posts

In [5]:
def fetch_all_posts(status_filter="open", limit_per_page=100):
    """Fetch all posts/questions from Metaculus API"""

    api_url = os.getenv("METACULUS_API_URL", "https://www.metaculus.com/api")
    api_token = os.getenv("METACULUS_API_TOKEN")
    db_path = os.getenv("DB_PATH")

    headers = {}
    if api_token and api_token != "YOUR_TOKEN_HERE":
        headers["Authorization"] = f"Token {api_token}"

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    all_posts = []
    next_url = f"{api_url}/posts/"
    params = {
        "limit": limit_per_page,
        "status": status_filter,
        "has_predictions": "true",
        "order_by": "-forecasters_count"
    }

    print(f"🔍 Fetching {status_filter} questions from Metaculus...")
    print("="*60)

    page = 0
    while next_url:
        try:
            page += 1
            response = requests.get(next_url, headers=headers, params=params if page == 1 else None)
            response.raise_for_status()

            data = response.json()
            results = data.get("results", [])

            if not results:
                break

            for post in results:
                # Store post in database
                cursor.execute("""
                INSERT OR REPLACE INTO posts
                (id, title, short_title, slug, author_id, author_username,
                 status, resolved, published_at, actual_close_time,
                 scheduled_close_time, scheduled_resolve_time, open_time,
                 nr_forecasters, comment_count, vote_score, forecasts_count)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    post.get('id'),
                    post.get('title'),
                    post.get('short_title'),
                    post.get('slug'),
                    post.get('author_id'),
                    post.get('author_username'),
                    post.get('status'),
                    post.get('resolved'),
                    post.get('published_at'),
                    post.get('actual_close_time'),
                    post.get('scheduled_close_time'),
                    post.get('scheduled_resolve_time'),
                    post.get('open_time'),
                    post.get('nr_forecasters'),
                    post.get('comment_count'),
                    post.get('vote', {}).get('score'),
                    post.get('forecasts_count')
                ))

                # Handle different question types
                if 'question' in post:
                    store_question(cursor, post['question'], post['id'])

                if 'conditional' in post:
                    store_conditional_question(cursor, post['conditional'], post['id'])

                if 'group_of_questions' in post:
                    store_group_questions(cursor, post['group_of_questions'], post['id'])

                all_posts.append(post)

            print(f"  📦 Page {page}: Fetched {len(results)} posts (Total: {len(all_posts)})")

            # Get next page URL
            next_url = data.get("next")

            # Rate limiting
            time.sleep(0.1)

        except Exception as e:
            print(f"❌ Error fetching posts: {e}")
            break

    conn.commit()
    conn.close()

    print("="*60)
    print(f"✅ Total posts fetched and stored: {len(all_posts)}")
    return all_posts

def store_question(cursor, question, post_id):
    """Store individual question data"""
    cursor.execute("""
    INSERT OR REPLACE INTO questions
    (id, post_id, title, type, status, resolution, resolution_criteria,
     fine_print, open_time, scheduled_close_time, actual_close_time,
     scheduled_resolve_time, actual_resolve_time, options, scaling, aggregation_data)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        question.get('id'),
        post_id,
        question.get('title'),
        question.get('type'),
        question.get('status'),
        question.get('resolution'),
        question.get('resolution_criteria'),
        question.get('fine_print'),
        question.get('open_time'),
        question.get('scheduled_close_time'),
        question.get('actual_close_time'),
        question.get('scheduled_resolve_time'),
        question.get('actual_resolve_time'),
        json.dumps(question.get('options')) if question.get('options') else None,
        json.dumps(question.get('scaling')) if question.get('scaling') else None,
        json.dumps(question.get('aggregations')) if question.get('aggregations') else None
    ))

def store_conditional_question(cursor, conditional, post_id):
    """Store conditional question data"""
    cursor.execute("""
    INSERT OR REPLACE INTO conditional_questions
    (id, parent_post_id, condition_id, condition_child_id,
     question_yes_id, question_no_id)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (
        conditional.get('id'),
        post_id,
        conditional.get('condition', {}).get('id'),
        conditional.get('condition_child', {}).get('id'),
        conditional.get('question_yes', {}).get('id'),
        conditional.get('question_no', {}).get('id')
    ))

    # Store the individual questions
    if 'condition' in conditional:
        store_question(cursor, conditional['condition'], post_id)
    if 'condition_child' in conditional:
        store_question(cursor, conditional['condition_child'], post_id)

def store_group_questions(cursor, group, post_id):
    """Store group of questions data"""
    cursor.execute("""
    INSERT OR REPLACE INTO group_questions
    (id, parent_post_id, group_variable, graph_type,
     description, resolution_criteria)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (
        group.get('id'),
        post_id,
        group.get('group_variable'),
        group.get('graph_type'),
        group.get('description'),
        group.get('resolution_criteria')
    ))

    # Store individual questions in the group
    for question in group.get('questions', []):
        store_question(cursor, question, post_id)

# Execute post fetching
posts = fetch_all_posts()

🔍 Fetching open questions from Metaculus...
  📦 Page 1: Fetched 100 posts (Total: 100)
  📦 Page 2: Fetched 100 posts (Total: 200)
  📦 Page 3: Fetched 100 posts (Total: 300)
  📦 Page 4: Fetched 100 posts (Total: 400)
  📦 Page 5: Fetched 100 posts (Total: 500)
  📦 Page 6: Fetched 100 posts (Total: 600)
  📦 Page 7: Fetched 100 posts (Total: 700)
  📦 Page 8: Fetched 100 posts (Total: 800)
  📦 Page 9: Fetched 100 posts (Total: 900)
  📦 Page 10: Fetched 100 posts (Total: 1000)
  📦 Page 11: Fetched 100 posts (Total: 1100)
  📦 Page 12: Fetched 100 posts (Total: 1200)
  📦 Page 13: Fetched 100 posts (Total: 1300)
  📦 Page 14: Fetched 100 posts (Total: 1400)
  📦 Page 15: Fetched 100 posts (Total: 1500)
  📦 Page 16: Fetched 100 posts (Total: 1600)
  📦 Page 17: Fetched 100 posts (Total: 1700)
  📦 Page 18: Fetched 100 posts (Total: 1800)
  📦 Page 19: Fetched 100 posts (Total: 1900)
  📦 Page 20: Fetched 100 posts (Total: 2000)
  📦 Page 21: Fetched 100 posts (Total: 2100)
  📦 Page 22: Fetched 100 post

In [6]:
def fetch_question_details(question_ids=None, sample_size=10):
    """Fetch detailed data for specific questions including aggregations"""

    api_url = os.getenv("METACULUS_API_URL", "https://www.metaculus.com/api")
    api_token = os.getenv("METACULUS_API_TOKEN")
    db_path = os.getenv("DB_PATH")

    headers = {}
    if api_token and api_token != "YOUR_TOKEN_HERE":
        headers["Authorization"] = f"Token {api_token}"

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # If no specific IDs provided, get sample of active questions
    if question_ids is None:
        cursor.execute("""
        SELECT q.id, q.title, q.type
        FROM questions q
        JOIN posts p ON q.post_id = p.id
        WHERE p.status = 'open' AND p.nr_forecasters > 10
        ORDER BY p.nr_forecasters DESC
        LIMIT ?
        """, (sample_size,))

        question_ids = [row[0] for row in cursor.fetchall()]

    print(f"📊 Fetching detailed data for {len(question_ids)} questions...")

    detailed_data = []
    for idx, q_id in enumerate(question_ids, 1):
        try:
            url = f"{api_url}/questions/{q_id}/"
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                data = response.json()

                # Store aggregation data if available
                if 'aggregations' in data:
                    for agg_type, agg_data in data['aggregations'].items():
                        if agg_data and 'latest' in agg_data and agg_data['latest']:
                            latest = agg_data['latest']
                            cursor.execute("""
                            INSERT INTO aggregations
                            (question_id, aggregation_type, start_time, end_time,
                             forecaster_count, centers, interval_lower_bounds,
                             interval_upper_bounds, forecast_values)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                            """, (
                                q_id,
                                agg_type,
                                latest.get('start_time'),
                                latest.get('end_time'),
                                latest.get('forecaster_count'),
                                json.dumps(latest.get('centers')),
                                json.dumps(latest.get('interval_lower_bounds')),
                                json.dumps(latest.get('interval_upper_bounds')),
                                json.dumps(latest.get('forecast_values'))
                            ))

                detailed_data.append(data)

                if idx % 10 == 0:
                    print(f"  📈 Progress: {idx}/{len(question_ids)} questions processed")

            time.sleep(0.1)  # Rate limiting

        except Exception as e:
            print(f"  ⚠️ Error fetching question {q_id}: {e}")

    conn.commit()
    conn.close()

    print(f"✅ Fetched detailed data for {len(detailed_data)} questions")
    return detailed_data

# Execute detailed fetching for top questions
detailed_questions = fetch_question_details(sample_size=20)

📊 Fetching detailed data for 20 questions...
✅ Fetched detailed data for 0 questions


In [7]:
def fetch_comments_for_posts(post_ids=None, limit=10):
    """Fetch comments for specified posts"""

    api_url = os.getenv("METACULUS_API_URL", "https://www.metaculus.com/api")
    api_token = os.getenv("METACULUS_API_TOKEN")
    db_path = os.getenv("DB_PATH")

    headers = {}
    if api_token and api_token != "YOUR_TOKEN_HERE":
        headers["Authorization"] = f"Token {api_token}"

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # If no specific IDs provided, get posts with most comments
    if post_ids is None:
        cursor.execute("""
        SELECT id, title, comment_count
        FROM posts
        WHERE status = 'open' AND comment_count > 0
        ORDER BY comment_count DESC
        LIMIT ?
        """, (limit,))

        post_data = cursor.fetchall()
        post_ids = [row[0] for row in post_data]

    print(f"💬 Fetching comments for {len(post_ids)} posts...")

    all_comments = []
    for idx, post_id in enumerate(post_ids, 1):
        try:
            url = f"{api_url}/comments/"
            params = {
                "post": post_id,
                "limit": 100
            }

            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                comments = data.get('results', [])

                for comment in comments:
                    cursor.execute("""
                    INSERT OR REPLACE INTO comments
                    (id, post_id, author_id, author_username, parent_id,
                     root_id, text, vote_score, included_forecast, created_at)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    """, (
                        comment.get('id'),
                        post_id,
                        comment.get('author', {}).get('id'),
                        comment.get('author', {}).get('username'),
                        comment.get('parent_id'),
                        comment.get('root_id'),
                        comment.get('text'),
                        comment.get('vote_score'),
                        comment.get('included_forecast'),
                        comment.get('created_at')
                    ))

                all_comments.extend(comments)

                if idx % 5 == 0:
                    print(f"  💭 Progress: {idx}/{len(post_ids)} posts processed")

            time.sleep(0.1)  # Rate limiting

        except Exception as e:
            print(f"  ⚠️ Error fetching comments for post {post_id}: {e}")

    conn.commit()
    conn.close()

    print(f"✅ Fetched {len(all_comments)} comments")
    return all_comments

# Execute comment fetching
comments = fetch_comments_for_posts(limit=10)

💬 Fetching comments for 10 posts...
  ⚠️ Error fetching comments for post 349: Error binding parameter 9: type 'dict' is not supported
  ⚠️ Error fetching comments for post 2534: Error binding parameter 9: type 'dict' is not supported
  ⚠️ Error fetching comments for post 353: Error binding parameter 9: type 'dict' is not supported
  💭 Progress: 5/10 posts processed
  ⚠️ Error fetching comments for post 384: Error binding parameter 9: type 'dict' is not supported
  ⚠️ Error fetching comments for post 18546: Error binding parameter 9: type 'dict' is not supported
  💭 Progress: 10/10 posts processed
✅ Fetched 500 comments


## Analyze Question Types and Statistics

In [8]:
def analyze_questions():
    """Generate analysis and statistics of fetched questions"""

    db_path = os.getenv("DB_PATH")
    conn = sqlite3.connect(db_path)

    print("\n" + "="*60)
    print("📊 METACULUS TRACKER - ANALYSIS REPORT")
    print("="*60)

    # Posts summary
    posts_df = pd.read_sql_query("""
        SELECT
            COUNT(*) as total_posts,
            SUM(CASE WHEN status = 'open' THEN 1 ELSE 0 END) as open_posts,
            SUM(CASE WHEN resolved = 1 THEN 1 ELSE 0 END) as resolved_posts,
            AVG(nr_forecasters) as avg_forecasters,
            SUM(forecasts_count) as total_forecasts
        FROM posts
    """, conn)

    print("\n📈 POSTS OVERVIEW:")
    print(f"   Total Posts: {posts_df['total_posts'].iloc[0]:,}")
    print(f"   Open Posts: {posts_df['open_posts'].iloc[0]:,}")
    print(f"   Resolved Posts: {posts_df['resolved_posts'].iloc[0]:,}")
    print(f"   Avg Forecasters per Post: {posts_df['avg_forecasters'].iloc[0]:.1f}")
    print(f"   Total Forecasts: {posts_df['total_forecasts'].iloc[0]:,}")

    # Question types breakdown
    questions_df = pd.read_sql_query("""
        SELECT
            type,
            COUNT(*) as count,
            AVG(CASE WHEN status = 'open' THEN 1 ELSE 0 END) as pct_open
        FROM questions
        GROUP BY type
        ORDER BY count DESC
    """, conn)

    print("\n🎯 QUESTION TYPES:")
    for idx, row in questions_df.iterrows():
        if row['type']:
            print(f"   {row['type'].capitalize()}: {row['count']:,} questions ({row['pct_open']*100:.1f}% open)")

    # Top posts by forecasters
    print("\n🔝 TOP 5 POSTS BY FORECASTERS:")
    top_posts = pd.read_sql_query("""
        SELECT title, nr_forecasters, forecasts_count, status
        FROM posts
        ORDER BY nr_forecasters DESC
        LIMIT 5
    """, conn)

    for idx, row in top_posts.iterrows():
        title = row['title'][:60] + "..." if len(row['title']) > 60 else row['title']
        print(f"   {idx+1}. {title}")
        print(f"      Forecasters: {row['nr_forecasters']:,} | Forecasts: {row['forecasts_count']:,} | Status: {row['status']}")

    # Recent activity
    recent_df = pd.read_sql_query("""
        SELECT
            DATE(published_at) as date,
            COUNT(*) as posts_published
        FROM posts
        WHERE published_at IS NOT NULL
        GROUP BY DATE(published_at)
        ORDER BY date DESC
        LIMIT 7
    """, conn)

    if len(recent_df) > 0:
        print("\n📅 RECENT PUBLISHING ACTIVITY (Last 7 days with posts):")
        for idx, row in recent_df.iterrows():
            print(f"   {row['date']}: {row['posts_published']} posts")

    # Aggregation statistics
    agg_stats = pd.read_sql_query("""
        SELECT
            COUNT(DISTINCT question_id) as questions_with_aggregations,
            COUNT(*) as total_aggregations,
            AVG(forecaster_count) as avg_forecasters_in_agg
        FROM aggregations
    """, conn)

    if agg_stats['total_aggregations'].iloc[0] > 0:
        print("\n📊 AGGREGATION DATA:")
        print(f"   Questions with Aggregations: {agg_stats['questions_with_aggregations'].iloc[0]:,}")
        print(f"   Total Aggregation Records: {agg_stats['total_aggregations'].iloc[0]:,}")
        print(f"   Avg Forecasters in Aggregations: {agg_stats['avg_forecasters_in_agg'].iloc[0]:.1f}")

    # Comments statistics
    comment_stats = pd.read_sql_query("""
        SELECT
            COUNT(*) as total_comments,
            COUNT(DISTINCT post_id) as posts_with_comments,
            COUNT(DISTINCT author_id) as unique_commenters,
            AVG(vote_score) as avg_vote_score
        FROM comments
    """, conn)

    if comment_stats['total_comments'].iloc[0] > 0:
        print("\n💬 COMMENT STATISTICS:")
        print(f"   Total Comments: {comment_stats['total_comments'].iloc[0]:,}")
        print(f"   Posts with Comments: {comment_stats['posts_with_comments'].iloc[0]:,}")
        print(f"   Unique Commenters: {comment_stats['unique_commenters'].iloc[0]:,}")
        print(f"   Avg Vote Score: {comment_stats['avg_vote_score'].iloc[0]:.2f}")

    print("\n" + "="*60)
    print(f"✅ Analysis complete!")
    print(f"📁 Database location: {db_path}")
    print("="*60)

    conn.close()

# Execute analysis
analyze_questions()


📊 METACULUS TRACKER - ANALYSIS REPORT

📈 POSTS OVERVIEW:
   Total Posts: 11,006
   Open Posts: 2,882
   Resolved Posts: 6,999
   Avg Forecasters per Post: 70.9
   Total Forecasts: 2,620,355

🎯 QUESTION TYPES:
   Binary: 8,675 questions (25.8% open)
   Numeric: 4,689 questions (27.0% open)
   Date: 1,170 questions (58.9% open)
   Multiple_choice: 671 questions (12.8% open)
   Discrete: 43 questions (62.8% open)

🔝 TOP 5 POSTS BY FORECASTERS:
   1. Will SpaceX land people on Mars before 2030?
      Forecasters: 4,382 | Forecasts: 7,975 | Status: open
   2. Who will be elected US President in 2024?
      Forecasters: 3,639 | Forecasts: 75,721 | Status: resolved
   3. Will there be a "World War Three" before 2050?
      Forecasters: 2,888 | Forecasts: 4,503 | Status: open
   4. Will there be Human-machine intelligence parity before 2040?
      Forecasters: 2,730 | Forecasts: 4,973 | Status: open
   5. Will the following Trump Cabinet nominees withdraw or be vot...
      Forecasters: 2,226

In [9]:
def monitor_high_activity_questions(threshold_forecasters=100, check_interval_seconds=60, duration_minutes=3):
    """Monitor questions with high forecasting activity"""

    db_path = os.getenv("DB_PATH")
    api_url = os.getenv("METACULUS_API_URL", "https://www.metaculus.com/api")
    api_token = os.getenv("METACULUS_API_TOKEN")

    headers = {}
    if api_token and api_token != "YOUR_TOKEN_HERE":
        headers["Authorization"] = f"Token {api_token}"

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Get high-activity questions
    cursor.execute("""
        SELECT p.id, p.title, p.nr_forecasters, p.forecasts_count
        FROM posts p
        WHERE p.status = 'open' AND p.nr_forecasters >= ?
        ORDER BY p.nr_forecasters DESC
        LIMIT 5
    """, (threshold_forecasters,))

    high_activity_posts = cursor.fetchall()

    if not high_activity_posts:
        print(f"❌ No questions found with {threshold_forecasters}+ forecasters")
        return

    print(f"📡 Monitoring {len(high_activity_posts)} high-activity questions")
    print(f"   Threshold: {threshold_forecasters}+ forecasters")
    print(f"   Duration: {duration_minutes} minutes")
    print(f"   Check interval: {check_interval_seconds} seconds")
    print("="*60)

    for post in high_activity_posts:
        title = post[1][:50] + "..." if len(post[1]) > 50 else post[1]
        print(f"   • {title}")
        print(f"     Forecasters: {post[2]:,} | Forecasts: {post[3]:,}")
    print("-"*60)

    start_time = datetime.now()
    end_time = start_time + pd.Timedelta(minutes=duration_minutes)
    check_count = 0

    print("🔄 Starting monitoring...")

    while datetime.now() < end_time:
        check_count += 1
        current_time = datetime.now()
        elapsed = (current_time - start_time).total_seconds()
        remaining = (end_time - current_time).total_seconds()

        print(f"\n⏱️ Check #{check_count} | Elapsed: {elapsed:.0f}s | Remaining: {remaining:.0f}s")

        for post_id, title, _, _ in high_activity_posts:
            try:
                # Fetch current data
                url = f"{api_url}/posts/{post_id}/"
                response = requests.get(url, headers=headers)

                if response.status_code == 200:
                    data = response.json()
                    new_forecasters = data.get('nr_forecasters', 0)
                    new_forecasts = data.get('forecasts_count', 0)

                    # Check for changes
                    cursor.execute("""
                        SELECT nr_forecasters, forecasts_count
                        FROM posts
                        WHERE id = ?
                    """, (post_id,))

                    old_data = cursor.fetchone()
                    if old_data:
                        old_forecasters, old_forecasts = old_data

                        if new_forecasters != old_forecasters or new_forecasts != old_forecasts:
                            print(f"   📈 UPDATE: {title[:40]}...")
                            print(f"      Forecasters: {old_forecasters} → {new_forecasters} ({new_forecasters - old_forecasters:+d})")
                            print(f"      Forecasts: {old_forecasts} → {new_forecasts} ({new_forecasts - old_forecasts:+d})")

                            # Update database
                            cursor.execute("""
                                UPDATE posts
                                SET nr_forecasters = ?, forecasts_count = ?
                                WHERE id = ?
                            """, (new_forecasters, new_forecasts, post_id))

            except Exception as e:
                print(f"   ⚠️ Error checking post {post_id}: {e}")

        conn.commit()

        # Wait for next check
        if datetime.now() < end_time:
            time.sleep(min(check_interval_seconds, (end_time - datetime.now()).total_seconds()))

    conn.close()

    print("\n" + "="*60)
    print("✅ Monitoring complete!")
    print(f"   Total checks: {check_count}")
    print(f"   Duration: {(datetime.now() - start_time).total_seconds():.1f} seconds")

# Execute monitoring (will run for 3 minutes)
monitor_high_activity_questions(threshold_forecasters=50, check_interval_seconds=30, duration_minutes=3)


📡 Monitoring 5 high-activity questions
   Threshold: 50+ forecasters
   Duration: 3 minutes
   Check interval: 30 seconds
   • Will SpaceX land people on Mars before 2030?
     Forecasters: 4,382 | Forecasts: 7,975
   • Will there be a "World War Three" before 2050?
     Forecasters: 2,888 | Forecasts: 4,503
   • Will there be Human-machine intelligence parity be...
     Forecasters: 2,730 | Forecasts: 4,973
   • Will any conclusive evidence for extraterrestrial ...
     Forecasters: 1,660 | Forecasts: 2,376
   • Will someone born before 2001 live to be 150?
     Forecasters: 1,572 | Forecasts: 3,026
------------------------------------------------------------
🔄 Starting monitoring...

⏱️ Check #1 | Elapsed: 0s | Remaining: 180s

⏱️ Check #2 | Elapsed: 31s | Remaining: 149s

⏱️ Check #3 | Elapsed: 63s | Remaining: 117s

⏱️ Check #4 | Elapsed: 94s | Remaining: 86s

⏱️ Check #5 | Elapsed: 125s | Remaining: 55s

⏱️ Check #6 | Elapsed: 157s | Remaining: 23s

✅ Monitoring complete!
   Total